# 82. NumPyスクラッチ実装（基礎編）

**Unit 0.4: 空間の帰納バイアス - CNN & 局所性の科学**

---

## このノートブックの位置づけ

前回学んだ畳み込みの数式を、実際にPythonコードとして実装します。あえて**forループを使った愚直な実装**から始めることで、計算過程を完全に理解します。

```
Unit 0.4 学習マップ
==================

Section A: 畳み込みの世界への入口
├── 80. 畳み込みとは何か（直感編）  ✅ 完了
├── 81. 畳み込みの数学的定義  ✅ 完了
├── 82. NumPyスクラッチ実装（基礎編）  ← 今ここ
├── 83. NumPyスクラッチ実装（高速化編）
├── 84. 古典的フィルタの解剖学
└── 85. カーネルの可視化と解釈
```

## 学習目標

このノートブックを終えると、以下のことができるようになります：

1. **forループで2D畳み込みを実装できる**
2. **各計算ステップを可視化して理解できる**
3. **パディング（境界処理）を実装できる**
4. **ストライドを実装できる**
5. **実装の計算量と問題点を説明できる**

## 目次

1. [学習目標](#学習目標)
2. [forループによる愚直な実装](#1-forループによる愚直な実装)
3. [境界処理（パディング）の実装](#2-境界処理パディングの実装)
4. [ストライドの実装](#3-ストライドの実装)
5. [動作確認：3x3平均フィルタ](#4-動作確認3x3平均フィルタ)
6. [課題：なぜこの実装は遅いのか](#5-課題なぜこの実装は遅いのか)

---

## 環境セットアップ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import time
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)
print("環境準備完了")

---

## 1. forループによる愚直な実装

まずは最もシンプルな実装から始めます。効率は度外視して、**計算過程が見える**ことを優先します。

### 1.1 2重ループの意味を図解

2D畳み込みの計算では、4重のループが必要です：

1. **出力の行** を走査（外側ループ1）
2. **出力の列** を走査（外側ループ2）
3. **カーネルの行** を走査（内側ループ1）
4. **カーネルの列** を走査（内側ループ2）

ただし、内側2つのループは `np.sum(window * kernel)` で置き換えられます。

In [ ]:
# ループ構造の図解

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 左：入力画像とカーネルの位置
ax1 = axes[0]
input_size = 5
kernel_size = 3

# 入力画像のグリッド
for i in range(input_size):
    for j in range(input_size):
        rect = Rectangle((j, input_size-1-i), 1, 1, 
                         fill=True, facecolor='lightblue', edgecolor='black', linewidth=1)
        ax1.add_patch(rect)
        ax1.text(j+0.5, input_size-0.5-i, f'({i},{j})', ha='center', va='center', fontsize=8)

# カーネル位置（出力(1,1)を計算中）
out_i, out_j = 1, 1
kernel_rect = Rectangle((out_j, input_size-1-out_i-kernel_size+1), kernel_size, kernel_size,
                        fill=False, edgecolor='red', linewidth=3)
ax1.add_patch(kernel_rect)

ax1.set_xlim(-0.5, input_size+0.5)
ax1.set_ylim(-0.5, input_size+0.5)
ax1.set_aspect('equal')
ax1.set_title(f'入力画像 ({input_size}×{input_size})\n赤枠: 出力({out_i},{out_j})計算時のカーネル位置', fontsize=11)
ax1.axis('off')

# 右：擬似コード
ax2 = axes[1]
ax2.axis('off')

pseudocode = """
【2D畳み込みの擬似コード】

def conv2d_naive(image, kernel):
    # 出力サイズを計算
    out_h = image_h - kernel_h + 1
    out_w = image_w - kernel_w + 1
    
    # 出力配列を初期化
    output = zeros(out_h, out_w)
    
    # 出力の各位置を走査 ← 外側ループ
    for i in range(out_h):          # 行方向
        for j in range(out_w):      # 列方向
            
            # カーネル範囲の入力を切り出し
            window = image[i:i+kernel_h, j:j+kernel_w]
            
            # 要素ごとの積の総和 ← 内側ループを置換
            output[i, j] = sum(window * kernel)
    
    return output


【計算量】
• 外側ループ: out_h × out_w 回
• 内側計算: kernel_h × kernel_w 回の乗算と加算
• 合計: O(out_h × out_w × kernel_h × kernel_w)
"""

ax2.text(0.05, 0.95, pseudocode, fontsize=10, family='monospace',
         verticalalignment='top', transform=ax2.transAxes,
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

### 1.2 コード1行ずつの動作確認

In [ ]:
def conv2d_naive_verbose(image, kernel):
    """
    2D畳み込み（相関）の愚直な実装
    計算過程を詳細に出力する教育用バージョン
    
    Parameters:
    -----------
    image : ndarray, shape (H, W)
        入力画像（2次元配列）
    kernel : ndarray, shape (kH, kW)
        畳み込みカーネル
    
    Returns:
    --------
    output : ndarray, shape (H-kH+1, W-kW+1)
        畳み込み結果
    """
    # Step 1: 入力の形状を取得
    image_h, image_w = image.shape
    kernel_h, kernel_w = kernel.shape
    print(f"Step 1: 形状取得")
    print(f"  入力画像: {image_h}×{image_w}")
    print(f"  カーネル: {kernel_h}×{kernel_w}")
    
    # Step 2: 出力サイズを計算
    out_h = image_h - kernel_h + 1
    out_w = image_w - kernel_w + 1
    print(f"\nStep 2: 出力サイズ計算")
    print(f"  出力高さ = {image_h} - {kernel_h} + 1 = {out_h}")
    print(f"  出力幅   = {image_w} - {kernel_w} + 1 = {out_w}")
    
    # Step 3: 出力配列を初期化
    output = np.zeros((out_h, out_w))
    print(f"\nStep 3: 出力配列を初期化 ({out_h}×{out_w})")
    
    # Step 4: メインループ
    print(f"\nStep 4: メインループ開始")
    print("="*50)
    
    for i in range(out_h):
        for j in range(out_w):
            # カーネル範囲の入力を切り出し
            window = image[i:i+kernel_h, j:j+kernel_w]
            
            # 要素ごとの積
            products = window * kernel
            
            # 総和
            result = np.sum(products)
            output[i, j] = result
            
            # 詳細出力（小さい例のみ）
            if out_h <= 4 and out_w <= 4:
                print(f"\n出力位置 ({i}, {j}):")
                print(f"  切り出し範囲: image[{i}:{i+kernel_h}, {j}:{j+kernel_w}]")
                print(f"  window =\n{window}")
                print(f"  kernel =\n{kernel}")
                print(f"  window * kernel =\n{products}")
                print(f"  sum = {result}")
    
    print("\n" + "="*50)
    print(f"\nStep 5: 完了")
    print(f"  出力配列:\n{output}")
    
    return output

In [ ]:
# 小さな例で動作確認

# 4×4の入力画像
test_image = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
], dtype=float)

# 2×2の簡単なカーネル（合計フィルタ）
test_kernel = np.array([
    [1, 1],
    [1, 1]
], dtype=float)

print("="*60)
print("2D畳み込みのステップバイステップ実行")
print("="*60)

result = conv2d_naive_verbose(test_image, test_kernel)

### 1.3 計算過程のステップバイステップ可視化

In [ ]:
def visualize_conv_step(image, kernel, position):
    """
    畳み込みの1ステップを可視化
    """
    i, j = position
    k_h, k_w = kernel.shape
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    # 1. 入力画像（カーネル位置をハイライト）
    ax1 = axes[0]
    ax1.imshow(image, cmap='Blues')
    for ii in range(image.shape[0]):
        for jj in range(image.shape[1]):
            ax1.text(jj, ii, f'{image[ii,jj]:.0f}', ha='center', va='center', fontsize=10)
    rect = Rectangle((j-0.5, i-0.5), k_w, k_h, 
                     fill=False, edgecolor='red', linewidth=3)
    ax1.add_patch(rect)
    ax1.set_title(f'入力画像\n赤枠: 位置({i},{j})の窓', fontsize=11)
    ax1.axis('off')
    
    # 2. 切り出した窓
    ax2 = axes[1]
    window = image[i:i+k_h, j:j+k_w]
    ax2.imshow(window, cmap='Blues')
    for ii in range(k_h):
        for jj in range(k_w):
            ax2.text(jj, ii, f'{window[ii,jj]:.0f}', ha='center', va='center', fontsize=12)
    ax2.set_title('切り出した窓', fontsize=11)
    ax2.axis('off')
    
    # 3. カーネル
    ax3 = axes[2]
    ax3.imshow(kernel, cmap='Oranges')
    for ii in range(k_h):
        for jj in range(k_w):
            ax3.text(jj, ii, f'{kernel[ii,jj]:.1f}', ha='center', va='center', fontsize=12)
    ax3.set_title('カーネル', fontsize=11)
    ax3.axis('off')
    
    # 4. 計算過程
    ax4 = axes[3]
    ax4.axis('off')
    
    products = window * kernel
    result = np.sum(products)
    
    calc_text = f"""【位置 ({i}, {j}) の計算】

窓 × カーネル（要素積）:
{np.array2string(products, precision=1)}

総和:
{' + '.join([f'{p:.1f}' for p in products.flatten()])}
= {result:.1f}

出力[{i}, {j}] = {result:.1f}
"""
    ax4.text(0.1, 0.9, calc_text, fontsize=10, family='monospace',
             verticalalignment='top', transform=ax4.transAxes,
             bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    ax4.set_title('計算過程', fontsize=11)
    
    plt.tight_layout()
    plt.show()
    
    return result

In [ ]:
# 各位置での計算を可視化

# 平均フィルタ
avg_kernel = np.ones((2, 2)) / 4

print("位置(0,0)での畳み込み計算:")
visualize_conv_step(test_image, avg_kernel, (0, 0))

print("\n位置(1,1)での畳み込み計算:")
visualize_conv_step(test_image, avg_kernel, (1, 1))

print("\n位置(2,2)での畳み込み計算:")
visualize_conv_step(test_image, avg_kernel, (2, 2))

### 1.4 シンプルな実装（本番用）

In [ ]:
def conv2d_naive(image, kernel):
    """
    2D畳み込み（相関）の愚直な実装
    パディングなし（Validモード）
    
    Parameters:
    -----------
    image : ndarray, shape (H, W)
        入力画像
    kernel : ndarray, shape (kH, kW)
        カーネル
    
    Returns:
    --------
    output : ndarray, shape (H-kH+1, W-kW+1)
    """
    # 形状を取得
    img_h, img_w = image.shape
    k_h, k_w = kernel.shape
    
    # 出力サイズ
    out_h = img_h - k_h + 1
    out_w = img_w - k_w + 1
    
    # 出力配列を初期化
    output = np.zeros((out_h, out_w))
    
    # メインループ
    for i in range(out_h):
        for j in range(out_w):
            # 窓を切り出して畳み込み計算
            window = image[i:i+k_h, j:j+k_w]
            output[i, j] = np.sum(window * kernel)
    
    return output

# 動作確認
print("シンプルな実装の動作確認:")
print(f"入力: {test_image.shape}, カーネル: {test_kernel.shape}")
result = conv2d_naive(test_image, test_kernel)
print(f"出力: {result.shape}")
print(f"結果:\n{result}")

---

## 2. 境界処理（パディング）の実装

実際のCNNでは、出力サイズを維持するためにパディングを使います。

### 2.1 np.pad の使い方

In [ ]:
# np.pad の基本的な使い方

original = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

print("元の配列:")
print(original)
print(f"形状: {original.shape}")

# パディング幅 = 1
pad_width = 1

# ゼロパディング
padded_zero = np.pad(original, pad_width, mode='constant', constant_values=0)
print(f"\nゼロパディング (pad={pad_width}):")
print(padded_zero)
print(f"形状: {padded_zero.shape}")

# 反射パディング
padded_reflect = np.pad(original, pad_width, mode='reflect')
print(f"\n反射パディング (pad={pad_width}):")
print(padded_reflect)

# エッジパディング
padded_edge = np.pad(original, pad_width, mode='edge')
print(f"\nエッジパディング (pad={pad_width}):")
print(padded_edge)

In [ ]:
# パディングの可視化

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

arrays = [
    (original, '元の配列', 'Blues'),
    (padded_zero, 'ゼロパディング', 'Blues'),
    (padded_reflect, '反射パディング', 'Blues'),
    (padded_edge, 'エッジパディング', 'Blues'),
]

for ax, (arr, title, cmap) in zip(axes, arrays):
    ax.imshow(arr, cmap=cmap)
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            # 元のデータ領域かどうかで色を変える
            is_original = (pad_width <= i < arr.shape[0]-pad_width and 
                          pad_width <= j < arr.shape[1]-pad_width) if arr.shape[0] > 3 else True
            color = 'black' if is_original else 'red'
            ax.text(j, i, f'{arr[i,j]:.0f}', ha='center', va='center', 
                   fontsize=11, color=color, fontweight='bold' if not is_original else 'normal')
    
    # 元の領域を枠で囲む
    if arr.shape[0] > 3:
        rect = Rectangle((pad_width-0.5, pad_width-0.5), 3, 3,
                         fill=False, edgecolor='green', linewidth=3)
        ax.add_patch(rect)
    
    ax.set_title(title, fontsize=12)
    ax.axis('off')

plt.suptitle('様々なパディング方法（赤字=パディング領域、緑枠=元データ）', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 2.2 パディング付き畳み込みの実装

In [ ]:
def conv2d_with_padding(image, kernel, padding=0, pad_mode='constant', pad_value=0):
    """
    パディング付き2D畳み込み
    
    Parameters:
    -----------
    image : ndarray, shape (H, W)
        入力画像
    kernel : ndarray, shape (kH, kW)
        カーネル
    padding : int
        パディング幅（上下左右に同じ幅）
    pad_mode : str
        パディングモード ('constant', 'reflect', 'edge')
    pad_value : float
        constantモード時の埋め値
    
    Returns:
    --------
    output : ndarray
    """
    # パディングを適用
    if padding > 0:
        if pad_mode == 'constant':
            padded = np.pad(image, padding, mode='constant', constant_values=pad_value)
        else:
            padded = np.pad(image, padding, mode=pad_mode)
    else:
        padded = image
    
    # 畳み込みを実行
    return conv2d_naive(padded, kernel)


def conv2d_same(image, kernel, pad_mode='constant'):
    """
    Sameパディング（出力サイズ = 入力サイズ）
    """
    k_h, k_w = kernel.shape
    
    # Sameパディングに必要なパディング幅
    # カーネルサイズが奇数の場合: padding = (k-1) // 2
    pad_h = (k_h - 1) // 2
    pad_w = (k_w - 1) // 2
    
    # パディング（非対称の場合は右下を多くする）
    pad_top = pad_h
    pad_bottom = k_h - 1 - pad_h
    pad_left = pad_w
    pad_right = k_w - 1 - pad_w
    
    if pad_mode == 'constant':
        padded = np.pad(image, ((pad_top, pad_bottom), (pad_left, pad_right)),
                       mode='constant', constant_values=0)
    else:
        padded = np.pad(image, ((pad_top, pad_bottom), (pad_left, pad_right)),
                       mode=pad_mode)
    
    return conv2d_naive(padded, kernel)

In [ ]:
# パディングの効果を確認

# テスト画像（5×5）
test_img = np.arange(25).reshape(5, 5).astype(float)
kernel_3x3 = np.ones((3, 3)) / 9  # 平均フィルタ

print("入力画像 (5×5):")
print(test_img)
print(f"\nカーネル (3×3 平均フィルタ):")
print(kernel_3x3.round(3))

# Validモード（パディングなし）
result_valid = conv2d_naive(test_img, kernel_3x3)
print(f"\nValid モード（パディングなし）:")
print(f"  出力サイズ: {result_valid.shape}")
print(f"  結果:\n{result_valid.round(2)}")

# Sameモード
result_same = conv2d_same(test_img, kernel_3x3)
print(f"\nSame モード（出力サイズ維持）:")
print(f"  出力サイズ: {result_same.shape}")
print(f"  結果:\n{result_same.round(2)}")

# 反射パディング
result_reflect = conv2d_same(test_img, kernel_3x3, pad_mode='reflect')
print(f"\nSame モード（反射パディング）:")
print(f"  出力サイズ: {result_reflect.shape}")
print(f"  結果:\n{result_reflect.round(2)}")

In [ ]:
# パディングモードの比較可視化

fig, axes = plt.subplots(2, 3, figsize=(14, 9))

# 上段：入力と各パディング結果
ax1 = axes[0, 0]
ax1.imshow(test_img, cmap='viridis')
ax1.set_title(f'入力画像 ({test_img.shape[0]}×{test_img.shape[1]})', fontsize=11)
ax1.axis('off')

ax2 = axes[0, 1]
ax2.imshow(result_valid, cmap='viridis')
ax2.set_title(f'Valid ({result_valid.shape[0]}×{result_valid.shape[1]})\n出力が縮小', fontsize=11)
ax2.axis('off')

ax3 = axes[0, 2]
ax3.imshow(result_same, cmap='viridis')
ax3.set_title(f'Same/ゼロパディング ({result_same.shape[0]}×{result_same.shape[1]})\n出力サイズ維持', fontsize=11)
ax3.axis('off')

# 下段：境界の比較
axes[1, 0].axis('off')
axes[1, 0].text(0.5, 0.5, '境界処理の\n違いに注目', fontsize=14, ha='center', va='center',
               transform=axes[1, 0].transAxes)

ax5 = axes[1, 1]
ax5.imshow(result_same, cmap='viridis')
for i in range(result_same.shape[0]):
    for j in range(result_same.shape[1]):
        ax5.text(j, i, f'{result_same[i,j]:.1f}', ha='center', va='center', fontsize=8,
                color='white' if result_same[i,j] < 15 else 'black')
ax5.set_title('ゼロパディング\n（境界が暗くなる）', fontsize=11)
ax5.axis('off')

ax6 = axes[1, 2]
ax6.imshow(result_reflect, cmap='viridis')
for i in range(result_reflect.shape[0]):
    for j in range(result_reflect.shape[1]):
        ax6.text(j, i, f'{result_reflect[i,j]:.1f}', ha='center', va='center', fontsize=8,
                color='white' if result_reflect[i,j] < 15 else 'black')
ax6.set_title('反射パディング\n（境界が自然）', fontsize=11)
ax6.axis('off')

plt.suptitle('パディングモードによる出力の違い（3×3平均フィルタ）', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n観察ポイント:")
print("- ゼロパディング：境界付近が元画像より暗くなる（0が混入するため）")
print("- 反射パディング：境界も内部と同様の値を維持")

---

## 3. ストライドの実装

ストライドを追加して、ダウンサンプリングを実現します。

In [ ]:
def conv2d_full(image, kernel, padding=0, stride=1, pad_mode='constant'):
    """
    パディングとストライドをサポートする2D畳み込み
    
    Parameters:
    -----------
    image : ndarray, shape (H, W)
        入力画像
    kernel : ndarray, shape (kH, kW)
        カーネル
    padding : int
        パディング幅
    stride : int
        ストライド（移動幅）
    pad_mode : str
        パディングモード
    
    Returns:
    --------
    output : ndarray
    """
    # パディングを適用
    if padding > 0:
        if pad_mode == 'constant':
            image = np.pad(image, padding, mode='constant', constant_values=0)
        else:
            image = np.pad(image, padding, mode=pad_mode)
    
    # 形状を取得
    img_h, img_w = image.shape
    k_h, k_w = kernel.shape
    
    # 出力サイズを計算（ストライド考慮）
    out_h = (img_h - k_h) // stride + 1
    out_w = (img_w - k_w) // stride + 1
    
    # 出力配列を初期化
    output = np.zeros((out_h, out_w))
    
    # メインループ（ストライド考慮）
    for i in range(out_h):
        for j in range(out_w):
            # ストライドを適用した位置
            row = i * stride
            col = j * stride
            
            # 窓を切り出して畳み込み計算
            window = image[row:row+k_h, col:col+k_w]
            output[i, j] = np.sum(window * kernel)
    
    return output

In [ ]:
# ストライドの効果を確認

# 8×8の入力画像
test_img_8x8 = np.arange(64).reshape(8, 8).astype(float)
kernel_3x3 = np.ones((3, 3)) / 9

print("入力画像 (8×8):")
print(test_img_8x8.astype(int))

# ストライド1
result_s1 = conv2d_full(test_img_8x8, kernel_3x3, padding=1, stride=1)
print(f"\nストライド=1, パディング=1:")
print(f"  出力サイズ: {result_s1.shape}")

# ストライド2
result_s2 = conv2d_full(test_img_8x8, kernel_3x3, padding=1, stride=2)
print(f"\nストライド=2, パディング=1:")
print(f"  出力サイズ: {result_s2.shape}")
print(f"  結果:\n{result_s2.round(1)}")

# ストライド4
result_s4 = conv2d_full(test_img_8x8, kernel_3x3, padding=1, stride=4)
print(f"\nストライド=4, パディング=1:")
print(f"  出力サイズ: {result_s4.shape}")
print(f"  結果:\n{result_s4.round(1)}")

In [ ]:
# ストライドの可視化

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

results = [
    (test_img_8x8, '入力 (8×8)'),
    (result_s1, 'stride=1 (8×8)'),
    (result_s2, 'stride=2 (4×4)'),
    (result_s4, 'stride=4 (2×2)'),
]

for ax, (img, title) in zip(axes, results):
    ax.imshow(img, cmap='viridis')
    ax.set_title(title, fontsize=12)
    ax.axis('off')

plt.suptitle('ストライドによるダウンサンプリング効果', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n出力サイズの公式確認:")
print(f"入力: 8, カーネル: 3, パディング: 1")
for s in [1, 2, 4]:
    out = (8 + 2*1 - 3) // s + 1
    print(f"  stride={s}: (8 + 2×1 - 3) / {s} + 1 = {out}")

---

## 4. 動作確認：3x3平均フィルタ

実際の画像に対して、実装した畳み込みを適用してみましょう。

In [ ]:
# テスト用の画像を作成

def create_test_image(size=64):
    """テスト用の合成画像を作成"""
    img = np.zeros((size, size))
    
    # 白い四角形
    img[15:35, 15:35] = 1.0
    
    # 白い円
    y, x = np.ogrid[:size, :size]
    center = (size*3//4, size*3//4)
    radius = size // 8
    mask = (x - center[0])**2 + (y - center[1])**2 <= radius**2
    img[mask] = 0.8
    
    # 斜めの線
    for i in range(size//4):
        if 45+i < size and 10+i < size:
            img[45+i, 10+i] = 0.6
            img[45+i, 11+i] = 0.6
    
    # ノイズを追加
    img += np.random.randn(size, size) * 0.1
    img = np.clip(img, 0, 1)
    
    return img

# 画像を作成
test_image = create_test_image(64)

plt.figure(figsize=(6, 6))
plt.imshow(test_image, cmap='gray')
plt.title('テスト画像 (64×64)', fontsize=12)
plt.colorbar()
plt.axis('off')
plt.show()

In [ ]:
# 様々なカーネルを適用

# カーネルの定義
kernels = {
    '平均 (3×3)': np.ones((3, 3)) / 9,
    '平均 (5×5)': np.ones((5, 5)) / 25,
    'ガウシアン': np.array([[1,2,1],[2,4,2],[1,2,1]]) / 16,
    'Sobel-X': np.array([[-1,0,1],[-2,0,2],[-1,0,1]]),
    'Sobel-Y': np.array([[-1,-2,-1],[0,0,0],[1,2,1]]),
    'Laplacian': np.array([[0,1,0],[1,-4,1],[0,1,0]]),
}

# 畳み込みを実行
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

# 元画像
axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=1)
axes[0].set_title('元画像', fontsize=11)
axes[0].axis('off')

# 各カーネルを適用
for ax, (name, kernel) in zip(axes[1:7], kernels.items()):
    # パディング計算
    pad = kernel.shape[0] // 2
    
    # 畳み込み実行
    result = conv2d_full(test_image, kernel, padding=pad, stride=1)
    
    # エッジ検出系は絶対値を取る
    if 'Sobel' in name or 'Laplacian' in name:
        result = np.abs(result)
        ax.imshow(result, cmap='gray')
    else:
        ax.imshow(result, cmap='gray', vmin=0, vmax=1)
    
    ax.set_title(name, fontsize=11)
    ax.axis('off')

# 最後のセルは空
axes[7].axis('off')

plt.suptitle('自作畳み込み関数による画像フィルタリング', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n各フィルタの効果:")
print("- 平均: ぼかし（周囲の平均）")
print("- ガウシアン: 自然なぼかし（中心重み付き）")
print("- Sobel-X: 水平方向のエッジ検出")
print("- Sobel-Y: 垂直方向のエッジ検出")
print("- Laplacian: 全方向のエッジ検出")

---

## 5. 課題：なぜこの実装は遅いのか

愚直な実装の問題点を理解し、次のノートブックへの動機付けとします。

### 5.1 計算時間の測定

In [ ]:
# 様々なサイズでの計算時間を測定

def measure_time(func, *args, n_runs=3):
    """関数の実行時間を測定"""
    times = []
    for _ in range(n_runs):
        start = time.time()
        result = func(*args)
        end = time.time()
        times.append(end - start)
    return np.mean(times), result

# テスト条件
sizes = [32, 64, 128, 256]
kernel = np.ones((3, 3)) / 9

print("画像サイズ vs 計算時間（3×3カーネル）")
print("="*50)

results = []
for size in sizes:
    img = np.random.rand(size, size)
    elapsed, _ = measure_time(conv2d_full, img, kernel, 1, 1)
    results.append((size, elapsed))
    print(f"  {size}×{size}: {elapsed*1000:.2f} ms")

print("\n※ 画像サイズが2倍になると、計算時間は約4倍になる（O(n²)）")

In [ ]:
# 計算時間のグラフ化

sizes_arr = np.array([r[0] for r in results])
times_arr = np.array([r[1] for r in results])

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 線形スケール
ax1 = axes[0]
ax1.plot(sizes_arr, times_arr * 1000, 'bo-', linewidth=2, markersize=10)
ax1.set_xlabel('画像サイズ (pixels)', fontsize=11)
ax1.set_ylabel('計算時間 (ms)', fontsize=11)
ax1.set_title('計算時間 vs 画像サイズ（線形スケール）', fontsize=12)
ax1.grid(True, alpha=0.3)

# 対数スケール
ax2 = axes[1]
ax2.loglog(sizes_arr, times_arr * 1000, 'bo-', linewidth=2, markersize=10, label='実測')

# 理論曲線（O(n²)）
theoretical = times_arr[0] * (sizes_arr / sizes_arr[0])**2 * 1000
ax2.loglog(sizes_arr, theoretical, 'r--', linewidth=2, label='O(n²) 理論')

ax2.set_xlabel('画像サイズ (pixels)', fontsize=11)
ax2.set_ylabel('計算時間 (ms)', fontsize=11)
ax2.set_title('計算時間 vs 画像サイズ（対数スケール）', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5.2 なぜ遅いのか：Pythonのforループの問題

In [ ]:
# Pythonのforループ vs NumPyのベクトル演算

size = 1000000  # 100万要素

a = np.random.rand(size)
b = np.random.rand(size)

# Pythonのforループ
def sum_loop(a, b):
    result = np.zeros_like(a)
    for i in range(len(a)):
        result[i] = a[i] + b[i]
    return result

# NumPyのベクトル演算
def sum_numpy(a, b):
    return a + b

# 時間測定
print(f"100万要素の配列の加算:")

start = time.time()
_ = sum_loop(a, b)
loop_time = time.time() - start
print(f"  Pythonループ: {loop_time*1000:.1f} ms")

start = time.time()
_ = sum_numpy(a, b)
numpy_time = time.time() - start
print(f"  NumPyベクトル: {numpy_time*1000:.3f} ms")

print(f"\n速度差: {loop_time/numpy_time:.0f}倍")

### 5.3 問題点のまとめ

In [ ]:
# 問題点のまとめ

summary = """
╔═══════════════════════════════════════════════════════════════════╗
║          愚直な畳み込み実装の問題点                              ║
╠═══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║  1. Pythonのforループは非常に遅い                                ║
║     - インタプリタのオーバーヘッド                               ║
║     - 各イテレーションで型チェックが発生                         ║
║                                                                   ║
║  2. メモリアクセスが非効率                                        ║
║     - image[i:i+k_h, j:j+k_w] で毎回新しいビューを作成          ║
║     - キャッシュ効率が悪い                                        ║
║                                                                   ║
║  3. 並列化されていない                                            ║
║     - 各出力位置は独立に計算可能                                  ║
║     - しかしPythonのGILがマルチスレッドを妨げる                  ║
║                                                                   ║
╠═══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║  解決策（次のノートブックで学習）:                                ║
║                                                                   ║
║  1. sliding_window_view でループを排除                           ║
║  2. im2col で行列積に変換                                        ║
║  3. PyTorch/CuPy でGPU並列化                                      ║
║                                                                   ║
╚═══════════════════════════════════════════════════════════════════╝
"""

print(summary)

---

## まとめ

### このノートブックで学んだこと

1. **forループによる2D畳み込みの実装**
   - 外側ループ：出力位置 (i, j) を走査
   - 内側計算：`np.sum(window * kernel)` で畳み込み

2. **パディングの実装**
   - `np.pad()` を使用
   - ゼロ / 反射 / エッジパディングの違い
   - Sameパディングで出力サイズ維持

3. **ストライドの実装**
   - `row = i * stride`, `col = j * stride` で位置を計算
   - ダウンサンプリング効果

4. **愚直な実装の問題点**
   - Pythonのforループは遅い
   - 大きな画像では実用的でない

---

## 次のステップ

次のノートブック **83. NumPyスクラッチ実装（高速化編）** では：

- `sliding_window_view` による高速化
- im2col テクニック（畳み込み → 行列積への変換）
- PyTorch の `conv2d` との速度比較

愚直な実装の100倍以上の高速化を実現する方法を学びます。